In [1]:
from endless_line.data_utils.dataloader import DataLoader

In [3]:
data = DataLoader(load_all_files=True)

In [20]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
load_dotenv(os.path.join(data.root_dir, '.secret'))

weather_api_key = os.getenv("OPENWEATHERMAP_API_KEY")

url = 'https://api.openweathermap.org/data/2.5/forecast?'
params = {
    'lat': 48.873492,
    'lon': 2.295104,
    'units': 'metric',
    'appid': weather_api_key
}

forecast = requests.get(url, params=params).json()
for stamp in forecast['list']:
	for info in stamp['main']:
		if info in ['sea_level', 'grnd_level', 'temp_min', 'temp_max', 'temp_kf']:
			continue
		stamp[info] = stamp['main'][info]
	match_description = {
		'clear sky': 'sky is clear',
		'few clouds: 11-25%': 'few clouds',
		'scattered clouds: 25-50%': 'scattered clouds',
		'broken clouds: 51-84%': 'broken clouds',
		'overcast clouds: 85-100%': 'overcast clouds',
	}
	for info in stamp['weather'][0]:
		if info in ['id']:
			continue
		elif info == 'description':
			stamp[f'weather_{info}'] = match_description.get(stamp['weather'][0]['description'], stamp['weather'][0]['description'])
		else:
			stamp[f'weather_{info}'] = stamp['weather'][0][info]

	stamp['clouds_all'] = stamp['clouds']['all']
	stamp['wind_speed'] = stamp['wind']['speed']

	keys_to_remove = {'sys', 'rain', 'weather', 'clouds', 'main', 'wind', 'pop', 'dt', 'visibility'}
	for key in keys_to_remove:
		stamp.pop(key, None)

forecast_df = pd.DataFrame(forecast['list'])
forecast_df['dt_txt'] = pd.to_datetime(forecast_df['dt_txt'], format='%Y-%m-%d %H:%M:%S')
forecast_df = forecast_df.set_index('dt_txt').resample('h').interpolate(method='ffill', limit_direction='forward')
forecast_df.reset_index(inplace=True)

In [4]:
data.weather

,dt_iso,temp,feels_like,pressure,wind_speed,rain_1h,rain_3h,clouds_all,weather_main,weather_description
166560,2018-01-01 00:00:00,8.94,5.21,1007,8.32,NaN,NaN,23,Clouds,few clouds
166561,2018-01-01 01:00:00,8.07,4.09,1008,8.26,NaN,NaN,4,Clear,sky is clear
166562,2018-01-01 02:00:00,7.76,3.81,1009,7.81,NaN,NaN,27,Clouds,scattered clouds
166563,2018-01-01 03:00:00,7.57,3.63,1010,7.59,NaN,NaN,78,Clouds,broken clouds
166564,2018-01-01 04:00:00,6.86,2.88,1010,7.06,NaN,NaN,52,Clouds,broken clouds
...,...,...,...,...,...,...,...,...,...,...
207259,2022-08-23 19:00:00,24.89,25.18,1016,1.34,NaN,NaN,100,Clouds,overcast clouds
207260,2022-08-23 20:00:00,23.06,23.35,1017,0.70,NaN,NaN,100,Clouds,overcast clouds
207261,2022-08-23 21:00:00,21.83,22.18,1017,0.70,NaN,NaN,100,Clouds,overcast clouds
207262,2022-08-23 22:00:00,21.08,21.46,1017,0.70,NaN,NaN,100,Clouds,overcast clouds


In [57]:
import pandas as pd


In [63]:
forecast_df = forecast_df.set_index('dt_txt').resample('h').interpolate(method='ffill', limit_direction='forward')
forecast_df.reset_index(inplace=True)

/var/folders/7p/tl03dfd12y384j_7fwmzm2vc0000gp/T/ipykernel_457/3989980723.py:1: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forecast_df = forecast_df.set_index('dt_txt').resample('h').interpolate(method='ffill', limit_direction='forward')


In [64]:
forecast_df.reset_index(inplace=True, drop=True)
forecast_df

,dt_txt,temp,feels_like,pressure,humidity,weather_main,weather_description,weather_icon,clouds_all,wind_speed
0,2025-02-10 18:00:00,5.59,3.96,1015.0,97.0,Rain,moderate rain,10n,100.0,2.10
1,2025-02-10 19:00:00,5.59,3.96,1015.0,97.0,Rain,light rain,10n,100.0,2.10
2,2025-02-10 20:00:00,5.07,1.88,1016.0,91.0,Rain,light rain,10n,100.0,4.12
3,2025-02-10 21:00:00,5.07,1.88,1016.0,91.0,Rain,light rain,10n,100.0,4.12
4,2025-02-10 22:00:00,5.07,1.88,1016.0,91.0,Clouds,overcast clouds,04n,100.0,4.12
...,...,...,...,...,...,...,...,...,...,...
113,2025-02-15 11:00:00,4.98,1.92,1025.0,47.0,Clear,sky is clear,01d,5.0,3.85
114,2025-02-15 12:00:00,4.98,1.92,1025.0,47.0,Clear,sky is clear,01d,5.0,3.85
115,2025-02-15 13:00:00,4.98,1.92,1025.0,47.0,Clear,sky is clear,01d,5.0,3.85
116,2025-02-15 14:00:00,6.25,3.17,1024.0,43.0,Clear,sky is clear,01d,1.0,4.44


In [3]:
data.clean_data()
data.weather.columns

Index(['dt_iso', 'temp', 'feels_like', 'pressure', 'wind_speed', 'rain_1h',
       'rain_3h', 'clouds_all', 'weather_main', 'weather_description'],
      dtype='object')

In [3]:
data.clean_weather()
data.weather

,dt_iso,temp,feels_like,pressure,wind_speed,rain_1h,rain_3h,clouds_all,weather_main,weather_description
166560,2018-01-01 00:00:00,8.94,5.21,1007,8.32,NaN,NaN,23,Clouds,few clouds
166561,2018-01-01 01:00:00,8.07,4.09,1008,8.26,NaN,NaN,4,Clear,sky is clear
166562,2018-01-01 02:00:00,7.76,3.81,1009,7.81,NaN,NaN,27,Clouds,scattered clouds
166563,2018-01-01 03:00:00,7.57,3.63,1010,7.59,NaN,NaN,78,Clouds,broken clouds
166564,2018-01-01 04:00:00,6.86,2.88,1010,7.06,NaN,NaN,52,Clouds,broken clouds
...,...,...,...,...,...,...,...,...,...,...
207259,2022-08-23 19:00:00,24.89,25.18,1016,1.34,NaN,NaN,100,Clouds,overcast clouds
207260,2022-08-23 20:00:00,23.06,23.35,1017,0.70,NaN,NaN,100,Clouds,overcast clouds
207261,2022-08-23 21:00:00,21.83,22.18,1017,0.70,NaN,NaN,100,Clouds,overcast clouds
207262,2022-08-23 22:00:00,21.08,21.46,1017,0.70,NaN,NaN,100,Clouds,overcast clouds


In [ ]:
from endless_line.data_utils.dataloader import DataLoader
data = DataLoader(load_all_files=True)
data.clean_data()

In [ ]:
attraction_df = data.link_attraction_park
attraction_df['ATTRACTION'].values


array(['Bumper Cars', 'Bungee Jump', 'Circus Train', 'Crazy Dance',
       'Dizzy Dropper', 'Drop Tower', 'Flying Coaster', 'Free Fall',
       'Giant Wheel', 'Giga Coaster', 'Go-Karts', 'Haunted House',
       'Himalaya Ride', 'Inverted Coaster', 'Kiddie Coaster',
       'Merry Go Round', 'Oz Theatre', 'Rapids Ride', 'Roller Coaster',
       'Spinning Coaster', 'Spiral Slide', 'Superman Ride', 'Swing Ride',
       'Vertical Drop', 'Water Ride', 'Zipline'], dtype=object)